In [11]:
from pandas.io.stata import StataReader
reader = StataReader('ships.dta')
dataframe = reader.data()

/Users/matthewmitchell/anaconda/lib/python2.7/site-packages/pandas/io/stata.py:1254: UserWarning: 'data' is deprecated, use 'read' instead
  warnings.warn("'data' is deprecated, use 'read' instead")


##Challenge 1

Model the damage incident counts with a Poisson Regression.

Hint: You can look at the previous ipython notebook with the logistic GLM example to see how you can do GLM with statsmodels
Remember that you will have to create dummy variables for categorical variables, and if you have time bins (like "1960-1964"), you have the option of either a) treating each bin as a category (and create dummies for each bin), or b) treat it as a continuous variable and take the mid-value (1962). Also remember to add a constant (to model the intercept).

Take a look at the statsmodels summary table, the goodness of fit indicators (Deviance, Pearson's chi square statistic) and the coefficients. Is this a good model?

In [35]:
dataframe.columns = ['t','construction','operation','months','damage']

In [57]:
dataframe.t = dataframe.t.astype(str)
dataframe.t = dataframe.t.replace('A',0)
dataframe.t = dataframe.t.replace('B',1)
dataframe.t = dataframe.t.replace('C',2)
dataframe.t = dataframe.t.replace('D',3)
dataframe.t = dataframe.t.replace('E',4)
dataframe.t = dataframe.t.astype(int)

dataframe.construction = dataframe.construction.astype(str)


In [67]:
dataframe.construction = dataframe.construction.astype(str)
dataframe['construction_start'], dataframe['construction_end'] = dataframe.construction.str.extract(r"""19(\d\d).*(\d\d)""")

,0,1
0,60,64
1,60,64
2,65,70
3,65,70
4,70,74
5,70,74
6,75,79
7,60,64
8,60,64
9,65,70


In [58]:
dataframe.head()

,t,construction,operation,months,damage
0,0,1960-64,1960-74,127,0
1,0,1960-64,1975-79,63,0
2,0,1965-70,1960-74,1095,3
3,0,1965-70,1975-79,1095,4
4,0,1970-74,1960-74,1512,6


In [61]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34 entries, 0 to 33
Data columns (total 5 columns):
t               34 non-null int64
construction    34 non-null category
operation       34 non-null category
months          34 non-null float32
damage          34 non-null float32
dtypes: category(2), float32(2), int64(1)
memory usage: 932.0 bytes


In [59]:
import statsmodels.api as sm
from patsy import dmatrices

In [62]:
y, X = dmatrices('damage ~ months + t', data=dataframe, return_type='dataframe')
model = sm.OLS(y, X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 damage   R-squared:                       0.722
Model:                            OLS   Adj. R-squared:                  0.704
Method:                 Least Squares   F-statistic:                     40.31
Date:                Mon, 08 Aug 2016   Prob (F-statistic):           2.38e-09
Time:                        19:23:26   Log-Likelihood:                -119.66
No. Observations:                  34   AIC:                             245.3
Df Residuals:                      31   BIC:                             249.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      5.6615      2.886      1.961      0.059        -0.225    11.548
months         0.0013      0.000      8.316      0.000         0.001     0.002
t             -0.8545      1.103     -0.774      0.445        -3.105     1.396
==============================================================================
Omnibus:                        8.318   Durbin-Watson:                   1.649
Prob(Omnibus):                  0.016   Jarque-Bera (JB):               14.437
Skew:                           0.241   Prob(JB):                     0.000733
Kurtosis:                       6.156   Cond. No.                     2.20e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.2e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""